In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import radarkit
import algorithms
import chart
import time

In [ ]:
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial', 'Helvetica']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rcParams['figure.figsize'] = (8.89, 5)   # Change the size of plots
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
# Read in data from netcdf files
sweep = radarkit.read('/Users/boonleng/Downloads/PX-20170220-050706-E2.4-Z.nc')
z = sweep['products']['Z']['data']
d = sweep['products']['D']['data']
p = sweep['products']['P']['data']
r = sweep['products']['R']['data']

In [ ]:
# Undo range correction and the ZCal in DSP to estimate the SNR
rr = 1.0e-3 * (np.arange(sweep['gateCount']) + 0.5) * sweep['gateSizeMeters']
snr = z - 20.0 * np.log10(rr) + 23.0

# Transition gate at 10.5 km at the given spacing
downSamplingRatio = int(sweep['gateSizeMeters'] / 30.0)
g = int((69.0e-6 * 3.0e8 * 0.5) / 30.0) + 5
g = int((g + downSamplingRatio - 1) / downSamplingRatio)

print(g)

z_off = 10.0 * np.log10(1.5 / 67)
snr[:, :g] = snr[:, :g] + z_off

# PhiDP calibration so that the transition is smooth and PhiDP starts ~ 0 deg
p[:, :g] = p[:, :g] + 40;
p[:, g:] = p[:, g:] + 44;

In [ ]:
gateSpacing = 1.0e-3 * sweep['gateSizeMeters']
maxRange = 0.9 * sweep['gateCount'] * gateSpacing

In [ ]:
tic = time.time()
zc, dc = algorithms.scwc(snr, z, d, p, r, dr=gateSpacing, verb=1)
toc = time.time()
print('Elapsed time = {} s'.format(toc - tic))

In [ ]:
# Original range (km) and zimuth (rad) of the data
rr, aa = np.meshgrid(1.0e-3 * sweep['range'], sweep['azimuth'] * (np.pi / 180.0))

# Cartesian coordinate of the radar cells
xx = rr * np.sin(aa)
yy = rr * np.cos(aa)

_ = chart.showPPI(xx, yy, z, style='Z', title='Reflectivity (dBZ)', maxrange=maxRange)
_ = chart.showPPI(xx, yy, zc, style='Z', title='Corrected Reflectivity (dBZ)', maxrange=maxRange)

_ = chart.showPPI(xx, yy, d, style='D', title='ZDR (dB)', maxrange=maxRange)
_ = chart.showPPI(xx, yy, dc, style='D', title='Corrected ZDR (dB)', maxrange=maxRange)